In [1]:
import os

os.startfile('c:\\')  # 파일 탐색기 열기

In [2]:
import webbrowser

webbrowser.open('https://freelec.co.kr/')

True

In [3]:
from subprocess import Popen

Popen('calc')  # 계산기 실행

In [4]:
Popen(['notepad', 'test.txt'])   # 메모장 실행

In [5]:
Popen([r"C:\Program Files\Microsoft Office\Office16\excel.exe"])

In [6]:
Popen([r"C:\Program Files\Microsoft Office\Office16\excel.exe", "weather.xlsx"])

In [7]:
from subprocess import Popen, PIPE

pipe = Popen('dir', shell=True, stdout=PIPE)
print(pipe.stdout.read().decode('cp949')[:284])

 E 드라이브의 볼륨: work
 볼륨 일련 번호: 1A94-8BC2

 E:\Projects\PythonBooks\업무자동화를위한파이썬 디렉터리

2019-01-02  오후 04:02    <DIR>          .
2019-01-02  오후 04:02    <DIR>          ..
2019-01-02  오후 03:26    <DIR>          .ipynb_checkpoints
2017-07-05  오후 03:01             1,315 00장-서론.ipynb


다음은 리눅스에서 실행한 예이다.

In [8]:
from subprocess import Popen, PIPE

pipe = Popen('ps -ef | grep gslee', shell=True, stdout=PIPE)
output = pipe.stdout.read().decode('utf-8')
for line in output.splitlines()[:5]:
    print(line)

gslee      676     1  0 11월12 ?      00:00:00 /usr/bin/ssh -q -oRSAAuthenticat
root       677  1196  0 11월12 ?      00:00:00 sshd: gslee [priv]  
gslee      717   677  0 11월12 ?      00:00:00 sshd: gslee@pts/54  
gslee      718   717  0 11월12 pts/54 00:00:00 -bash
gslee      732     1  0 11월12 ?      00:00:00 /usr/bin/ssh -q -oRSAAuthenticat


만일 프로그램 실행 후 입력을 제공하고, 출력 결과를 받는 것을 모두 해야 한다면 양방향 파이프를 이용한다. 입력은 `stdin`에, 출력은 `stdout`에 연결한다. 입력과 출력의 자료형은 모두 바이트형이다. 

<span style="color:red">/** 참고 **/</span>

문자열을 바이트 열로 변환하려면 `encode(code)` 메서드를 이용한다.

In [9]:
'문자열'.encode('utf-8')

b'\xeb\xac\xb8\xec\x9e\x90\xec\x97\xb4'

바이트 열을 문자열로 변환하려면 `str()` 함수에 코드를 제공한다.

In [10]:
str(b'\xeb\xac\xb8\xec\x9e\x90\xec\x97\xb4', 'utf-8')

'문자열'

<span style="color:red">/** 참고 끝 **/</span>

In [11]:
from subprocess import Popen, PIPE

pipe = Popen('sort', shell=True, stdin=PIPE, stdout=PIPE)
pipe.stdin.write(b'''a
b
x
c''')
pipe.stdin.close()

In [12]:
print(str(pipe.stdout.read(), 'cp949'))

a
b
c
x



## 도전 과제 - 지정된 시간에 함수 실행하기(블로킹 모드)


In [13]:
import sys
import sched

def my_job(text="..."):
    print(text)
    sys.stdout.flush()
    
s = sched.scheduler()
s.enter(5, 1, my_job)
s.enter(10, 1, my_job, argument=('good!',))
s.run()  # Blocking
print('done')

...
good!
done


## 도전과제 - threading을 이용한 지정된 시간에 함수 실행하기(논블로킹 모드)


In [14]:
import threading
import time

def my_job(text):
    print(text)
    
threading.Timer(5, my_job, ('good!!',)).start()   # 5초 대기 후 실행
print('other works continued..')
time.sleep(5)

other works continued..
good!!


지정된 시간에 함수가 실행되도록 위의 코드를 확장 해보자.

In [15]:
import threading
import time

def run_it_at(at, func, args=()):
    t = time.strptime(at, '%Y-%m-%d %H:%M:%S')
    t = time.mktime(t)          # 지정된 시간을 숫자 시간으로 변환
    diff_sec = t - time.time()  # 현재 시간과의 차이 => 대기시간(초)
    print(diff_sec)
    threading.Timer(diff_sec, func, args).start() 
    
def my_job(text):
    print(text)
    
run_it_at('2019-01-02 20:22:00', my_job, ('good!!!',))
print('other works continued..')
time.sleep(5)

40.09665274620056
other works continued..
good!!!


## 도전과제 - 주기적으로 함수 실행하기

In [16]:
import schedule
import time

def job(msg="I'm working..."):
    print(msg)

schedule.every().seconds.do(job)  # 1초 마다
schedule.every(5).seconds.do(job, ('5 sec',))  # 매 5초마다
schedule.every(10).minutes.do(job) # 10분 마다
schedule.every().hour.do(job, ('hour',))  # 매 시간
schedule.every().day.at("10:30").do(job)  # 10:30분에
schedule.every(5).to(10).seconds.do(job) # 5초에서 10초 마다 (랜덤으로 선택된다)
schedule.every().wednesday.at("13:15").do(job)  # 수요일 13:15 마다

while True:
    schedule.run_pending()  # 대기 중인 작업이 있다면 처리한다.
    time.sleep(1)

## 도전 과제 - RSS에서 주기적으로 정보 수집

In [17]:
import smtplib
import feedparser
from email.mime.text import MIMEText
from email.header import Header

# 관심 RSS feed 주소를 추가, 변경한다.
rss_feeds = [
    'http://file.mk.co.kr/news/rss/rss_30100041.xml',
    'http://file.mk.co.kr/news/rss/rss_50300009.xml'
]

# words of interest. 관심 단어
WOI = ['금리', '대출', '가격']

out = []
for feed in rss_feeds:
    d = feedparser.parse(feed)
    for entry in d.entries:
        for w in WOI:  # 관심 단어가 제목이나 요약에 있는지 살핀다
            if w in entry['title'] or w in entry['summary']:
                s = '* {} {}'.format(entry['title'], entry['link'])
                out.append(s)
                break
message = '\n'.join(out)

if msg:
    # 메일 발송
    subject = 'RSS News'
    mail_from = mail_to = 'my_id@gmail.com'
    id_ = 'my_id'
    pw_ = 'my_password'

    smtp = smtplib.SMTP('smtp.gmail.com', 587)
    smtp.ehlo()
    smtp.starttls()
    smtp.login(id_, pw_)
    
    msg = MIMEText(message.encode('utf-8'), _subtype='plain', _charset='utf-8')
    msg['Subject'] = Header(subject.encode('utf-8'), 'utf-8')
    msg['From'] = mail_from
    msg['To'] = mail_to

    smtp.sendmail(mail_from, mail_to, msg.as_string( ))
    smtp.quit()    